In [1]:
# =====================================================
# FAKE NEWS DETECTION – SVM (LinearSVC) + GonzaloA/fake_news
# Output Format: Dictionary-style
# =====================================================

# 1. CÀI ĐẶT & IMPORT
# !pip install -q datasets scikit-learn pandas numpy

import re
import time
import warnings
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

# 2. TẢI DATASET
print("\nĐang tải dataset GonzaloA/fake_news...")
dataset = load_dataset("GonzaloA/fake_news")

# 3. XỬ LÝ DỮ LIỆU
def process_data(ds_split):
    df = pd.DataFrame(ds_split)
    # Ghép title + text
    df['content'] = df.get('title', '').fillna('') + " " + df.get('text', '').fillna('')
    return df

if 'validation' in dataset and 'test' in dataset:
    print("Sử dụng các tập train/val/test có sẵn.")
    train_df = process_data(dataset['train'])
    test_df = process_data(dataset['test'])
    # Gộp val vào train để tối ưu cho SVM
    val_df = process_data(dataset['validation'])
    train_df = pd.concat([train_df, val_df], ignore_index=True)
else:
    print("Tự chia tập dữ liệu...")
    df = process_data(dataset['train'])
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Hàm làm sạch văn bản
def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("Đang làm sạch văn bản...")
train_df['clean_content'] = train_df['content'].apply(clean_text)
test_df['clean_content'] = test_df['content'].apply(clean_text)

# Lọc mẫu ngắn
train_df = train_df[train_df['clean_content'].str.len() > 20]
test_df = test_df[test_df['clean_content'].str.len() > 20]

print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")

# 4. TF-IDF VECTORIZATION
print("\nĐang tạo vector TF-IDF...")
vectorizer = TfidfVectorizer(
    max_features=50000,
    ngram_range=(1, 2),
    stop_words='english'
)

X_train = vectorizer.fit_transform(train_df['clean_content'])
y_train = train_df['label']

X_test = vectorizer.transform(test_df['clean_content'])
y_test = test_df['label']

# 5. HUẤN LUYỆN SVM (LinearSVC + Calibration)
print("\nBắt đầu huấn luyện SVM...")
# LinearSVC chạy nhanh hơn SVC rất nhiều
linear_svc = LinearSVC(class_weight='balanced', random_state=42, max_iter=1000)

# CalibratedClassifierCV giúp SVM có thể dự đoán xác suất (predict_proba) để tính AUC
clf = CalibratedClassifierCV(linear_svc)
clf.fit(X_train, y_train)

# 6. ĐÁNH GIÁ & ĐO THỜI GIAN
print("\nĐang đánh giá trên tập Test...")

start_time = time.time()
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1] # Lấy xác suất lớp Positive (Fake/Real)
end_time = time.time()

# Tính toán các chỉ số
eval_runtime = end_time - start_time
n_samples = len(y_test)
eval_samples_per_second = n_samples / eval_runtime

acc = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
auc = roc_auc_score(y_test, y_prob)

# Tạo dictionary kết quả
results = {
    'eval_accuracy': acc,
    'eval_precision': precision,
    'eval_recall': recall,
    'eval_f1': f1,
    'eval_auc': float(auc),
    'eval_loss': 'N/A (SVM)',
    'eval_runtime': eval_runtime,
    'eval_samples_per_second': eval_samples_per_second,
    'eval_steps_per_second': 'N/A'
}

print("-" * 30)
print("SVM_LinearSVC_GonzaloA_FakeNews")
print(results)
print("-" * 30)


Đang tải dataset GonzaloA/fake_news...


README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/38.8M [00:00<?, ?B/s]

...

data/test-00000-of-00001.parquet:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24353 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8117 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8117 [00:00<?, ? examples/s]

Sử dụng các tập train/val/test có sẵn.
Đang làm sạch văn bản...
Train size: 32465
Test size: 8117

Đang tạo vector TF-IDF...

Bắt đầu huấn luyện SVM...

Đang đánh giá trên tập Test...
------------------------------
SVM_LinearSVC_GonzaloA_FakeNews
{'eval_accuracy': 0.9781939140076383, 'eval_precision': 0.9782085870352637, 'eval_recall': 0.9781939140076383, 'eval_f1': 0.9781969430003693, 'eval_auc': 0.9965554679270532, 'eval_loss': 'N/A (SVM)', 'eval_runtime': 0.05560135841369629, 'eval_samples_per_second': 145985.64192634076, 'eval_steps_per_second': 'N/A'}
------------------------------
